In [1]:
import sqlite3
sqlite3.sqlite_version
import pandas as pd
# Create a connection with database
conn = sqlite3.connect("F:\Machine learning\SQL\imdb.db")

In [2]:
# Load all the table
table = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'table'", conn)

In [3]:
table

,type,name,tbl_name,rootpage,sql
0,table,Movie,Movie,2,"CREATE TABLE ""Movie"" (\n""index"" INTEGER,\n ""M..."
1,table,Genre,Genre,4,"CREATE TABLE ""Genre"" (\n""index"" INTEGER,\n ""N..."
2,table,Language,Language,5,"CREATE TABLE ""Language"" (\n""index"" INTEGER,\n ..."
3,table,Country,Country,6,"CREATE TABLE ""Country"" (\n""index"" INTEGER,\n ..."
4,table,Location,Location,7,"CREATE TABLE ""Location"" (\n""index"" INTEGER,\n ..."
5,table,M_Location,M_Location,11,"CREATE TABLE ""M_Location"" (\n""index"" INTEGER,\..."
6,table,M_Country,M_Country,10,"CREATE TABLE ""M_Country"" (\n""index"" INTEGER,\n..."
7,table,M_Language,M_Language,9,"CREATE TABLE ""M_Language"" (\n""index"" INTEGER,\..."
8,table,M_Genre,M_Genre,8,"CREATE TABLE ""M_Genre"" (\n""index"" INTEGER,\n ..."
9,table,Person,Person,12,"CREATE TABLE ""Person"" (\n""index"" INTEGER,\n ""..."


In [4]:
print("Type: table\n")
print("Number of unique names:", table['name'].nunique(), '\n')
print("Number of unique table names:", table['tbl_name'].nunique(), '\n')
print("Unique names:\n")
print(table['name'].unique())

Type: table

Number of unique names: 13 

Number of unique table names: 13 

Unique names:

['Movie' 'Genre' 'Language' 'Country' 'Location' 'M_Location' 'M_Country'
 'M_Language' 'M_Genre' 'Person' 'M_Producer' 'M_Director' 'M_Cast']


In [5]:
print("\nDistinct tables are\n")

pd.read_sql_query("SELECT DISTINCT tbl_name FROM sqlite_master WHERE type = 'table'", conn)


Distinct tables are



,tbl_name
0,Movie
1,Genre
2,Language
3,Country
4,Location
5,M_Location
6,M_Country
7,M_Language
8,M_Genre
9,Person


## Assignment

### 1.List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [8]:
rows = pd.read_sql_query('''select Name,title,year from Person p join M_Director md on p.PID = md.PID join Movie m on m.MID = md.MID where m.MID in
                            (select MID from Movie where (year%4=0 and year%100!=0) and (year%4=0 or year%100=0 and year%400=0) and MID in
                            (select MID from M_Genre where GID in(select GID from Genre where Name LIKE '%comedy%')))''', conn)
print(rows.head(10))

              Name                              title  year
0     Milap Zaveri                         Mastizaade  2016
1     Milap Zaveri                         Mastizaade  2016
2     Danny Leiner  Harold & Kumar Go to White Castle  2004
3     Danny Leiner  Harold & Kumar Go to White Castle  2004
4   Anurag Kashyap                 Gangs of Wasseypur  2012
5   Anurag Kashyap                 Gangs of Wasseypur  2012
6     Frank Coraci        Around the World in 80 Days  2004
7     Frank Coraci        Around the World in 80 Days  2004
8    Griffin Dunne             The Accidental Husband  2008
9    Griffin Dunne             The Accidental Husband  2008


### 2. List the names of all the actors who played in the movie 'Anand' (1971)

In [9]:
Movie_Anand_Actors = 'select person.name from movie \
inner join m_cast on movie.mid = m_cast.mid \
inner join person on trim(m_cast.pid) = person.pid \
where movie.title = "Anand"'
pd.read_sql_query(Movie_Anand_Actors, conn)

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Brahm Bhardwaj
3,Ramesh Deo
4,Seema Deo
5,Dev Kishan
6,Durga Khote
7,Lalita Kumari
8,Lalita Pawar
9,Atam Prakash


### 3) List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [10]:
Actors_1970_1990 = 'select person.name from person \
where pid in (select distinct trim(m_cast.pid) from \
movie inner join m_cast on movie.mid = m_cast.mid where movie.year < 1970) \
and person.pid in (select distinct trim(m_cast.pid) from \
movie inner join m_cast on movie.mid = m_cast.mid where movie.year > 1990)'

pd.read_sql_query(Actors_1970_1990, conn)


,Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni
...,...
348,Vinod Mehra
349,Deven Verma
350,Master Bhagwan
351,Rishi Kapoor


### 4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [11]:
Directors_Movies_More_Than_10 = 'SELECT distinct person.name, SQ.COUNT \
FROM person JOIN \
( select m_director.pid, count(m_director.mid) as COUNT from m_director \
group by m_director.pid having count(m_director.mid) >= 10 \
) AS SQ ON trim(person.pid) = trim(SQ.pid) group by SQ.pid order by SQ.COUNT desc'

pd.read_sql_query(Directors_Movies_More_Than_10, conn)


,Name,COUNT
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Priyadarshan,30
3,Ram Gopal Varma,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Basu Chatterjee,19
8,Shakti Samanta,19
9,Subhash Ghai,18


### 5.a) For each year, count the number of movies in that year that had only female actors.

In [12]:
Female_Actors = 'select m.year, count(*) \
from movie m \
where not exists (select * from person p, m_cast c where p.pid = trim(c.pid) and c.mid = m.mid and \
p.gender !="Female") \
group by m.year'

pd.read_sql_query(Female_Actors, conn)


,year,count(*)
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1
5,2018,1
6,I 2018,1


### 5.b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer

In [25]:
Female_Movie_Percentage = pd.read_sql_query("""

                                            WITH
                
                                            Movie_Non_Females AS
                                            (SELECT DISTINCT TRIM(MC.MID) MID
                                            FROM M_CAST MC
                                            JOIN PERSON P ON TRIM(MC.PID) = TRIM(P.PID)
                                            WHERE TRIM(P.GENDER) IN ('Male','None')),

                                            MOVIE_FEMALE_Year AS
                                            (SELECT CAST(SUBSTR(M.YEAR,-4) AS UNASSIGNED) YEAR,
                                            COUNT(DISTINCT TRIM(MID)) Female_Movie_Only
                                            FROM Movie M
                                            WHERE TRIM(MID) NOT IN (SELECT MID FROM Movie_Non_Females)
                                            GROUP BY CAST(SUBSTR(M.year,-4) AS UNASSIGNED)),

                                            MOVIES_YEAR AS
                                            (SELECT CAST(SUBSTR(M.YEAR,-4) AS UNASSIGNED) Year,
                                            COUNT(DISTINCT TRIM(MID)) Total_Movies
                                            FROM MOVIE M
                                            GROUP BY CAST(SUBSTR(M.YEAR,-4) AS UNASSIGNED))

                                            SELECT MY.YEAR, MY.Total_Movies,
                                            ROUND((IFNULL(MF.Female_Movie_Only, 0) * 100)/MY.Total_Movies, 2) Female_Movie_Percentage
                                            FROM MOVIES_YEAR MY
                                            LEFT OUTER JOIN MOVIE_FEMALE_Year MF ON
                                            TRIM(MY.YEAR) = TRIM(MF.YEAR)
                                            ORDER BY Female_Movie_Percentage DESC
                
                                            """, conn)

print("\nPercentage of movies with only females")
print('-'*38, '\n')
print(Female_Movie_Percentage)


Percentage of movies with only females
-------------------------------------- 

    Year  Total_Movies  Female_Movie_Percentage
0   1939             2                     50.0
1   1999            66                      1.0
2   2000            64                      1.0
3   2018           104                      1.0
4   1931             1                      0.0
..   ...           ...                      ...
73  2013           136                      0.0
74  2014           126                      0.0
75  2015           119                      0.0
76  2016           129                      0.0
77  2017           126                      0.0

[78 rows x 3 columns]


### 6) Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [14]:
Largest_Cast  = 'select movie.title as TITLE, count(m_cast.pid) as CAST_COUNT \
from movie inner join m_cast on movie.mid = m_cast.mid group by movie.mid \
order by CAST_COUNT desc LIMIT 1'

pd.read_sql_query(Largest_Cast , conn)

,TITLE,CAST_COUNT
0,Ocean's Eight,238


### A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.


In [17]:
Decade_More_Movies  = 'select y.year as decade_start, y.year + 9 as decade_end, count(*) as num_movies from (select distinct year from movie) y join movie m on m.year >= y.year and m.year < y.year + 10 group by y.year  order by count(*) desc  limit 1'

pd.read_sql_query(Decade_More_Movies , conn)

,decade_start,decade_end,num_movies
0,2008,2017,1128


###  Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies.

In [18]:
Actors_Never_Unemployed_More_3  = 'select Name from Person \
where PID not in (select distinct(pid)from m_cast as c1 natural join movie as m1 \
where exists \
(select mid from m_cast as c2 natural join movie as m2 \
where c1.pid = c2.pid and (m2.year - 3) > m1.year \
and not exists(select mid from m_cast as c3 natural join movie as m3 \
where c1.pid = c3.pid and m1.year < m3.year and m3.year < m2.year)))'

pd.read_sql_query(Actors_Never_Unemployed_More_3, conn)

,Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
...,...
38280,Kannan
38281,Adrian Fulle
38282,Gulshan Kumar
38283,Iqbal


### Find all the actors that made more movies with Yash Chopra than any other director

In [3]:
Actors_more_movie_Yash = pd.read_sql_query("""

                                WITH
                                
                                YASH_CHOPRA AS
                                (SELECT TRIM(P.PID) PID
                                FROM PERSON P
                                WHERE Trim(P.NAME) = 'Yash Chopra'),
                                
                                MOVIES_COUNT AS
                                (SELECT TRIM(MC.PID) ACTORS, TRIM(MD.PID) DIRECTORS,
                                COUNT(DISTINCT TRIM(MD.MID)) MOVIE_COUNT
                                FROM M_CAST MC, M_DIRECTOR MD
                                WHERE TRIM(MC.MID) = TRIM(MD.MID)
                                GROUP BY ACTORS, DIRECTORS),
    
                                YASH_MOVIE_COUNT AS
                                (SELECT CM.ACTORS, CM.DIRECTORS,
                                CM.MOVIE_COUNT MOVIE_COUNT_YASH
                                FROM MOVIES_COUNT CM, YASH_CHOPRA YC
                                WHERE CM.DIRECTORS = YC.PID),
    
                                OTHER_DIRECTORS AS
                                (SELECT ACTORS, MAX(MOVIE_COUNT) MAX_MOVIE_COUNT
                                FROM MOVIES_COUNT CM, YASH_CHOPRA YC
                                WHERE CM.DIRECTORS <> YC.PID 
                                GROUP BY ACTORS),
    
                                ACTORS_MOVIE AS
                                (SELECT YM.ACTORS,
                                CASE WHEN YM.MOVIE_COUNT_YASH > IFNULL(OD.MAX_MOVIE_COUNT, 0) THEN 'YES' ELSE 'NO' END MAX_YASH_MOVIE
                                FROM YASH_MOVIE_COUNT YM
                                LEFT OUTER JOIN OTHER_DIRECTORS OD ON YM.ACTORS = OD.ACTORS)
    
                                SELECT DISTINCT TRIM(P.NAME) NAME_OF_ACTORS
                                FROM PERSON P
                                WHERE TRIM(P.PID) IN (SELECT DISTINCT ACTORS
                                FROM ACTORS_MOVIE
                                WHERE MAX_YASH_MOVIE = 'YES')
            
                                """, conn)

print("\nActors who worked more in movies directed by Yash:")
print('-'*50, '\n')
print(Actors_more_movie_Yash)


Actors who worked more in movies directed by Yash:
-------------------------------------------------- 

         NAME_OF_ACTORS
0        Waheeda Rehman
1        Achala Sachdev
2           Yash Chopra
3            Vinod Negi
4    Chandni Jas Keerat
..                  ...
100          Yasin Khan
101     Sandow S. Sethi
102               Naval
103           Prem Sood
104     Ramlal Shyamlal

[105 rows x 1 columns]


### The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [21]:
Shahrukh_Actors  = 'select distinct name from person \
inner join m_cast on trim(m_cast.pid) = person.pid \
where name not like "%Shah Rukh Khan" and mid in \
(select mid from m_cast where trim(pid) in (select person.pid from person inner join m_cast on trim(m_cast.pid) = person.pid \
where name not like "%Shah Rukh Khan" and mid in \
(select mid from person inner join m_cast on trim(m_cast.pid) = person.pid \
where name like "%Shah Rukh Khan"))) and person.pid not in \
(select person.pid from person inner join m_cast on trim(m_cast.pid) = person.pid \
where name not like "%Shah Rukh Khan" and mid in (select mid from person inner join m_cast on trim(m_cast.pid) = person.pid \
where name like "%Shah Rukh Khan"))'

pd.read_sql_query(Shahrukh_Actors , conn)

,Name
0,Alicia Vikander
1,Dominic West
2,Walton Goggins
3,Daniel Wu
4,Kristin Scott Thomas
...,...
24586,Minoo Mehtab
24587,Hayley Cleghorn
24588,Nirvasha Jithoo
24589,Kamal Maharshi
